In [1]:
from lib import *
import numpy as np

import os
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

from plotly.subplots import make_subplots
from itertools import product

*** THIS NOTEBOOK CAN BE USED TO ANALYSE RAW AND DEC OP.WVFs FROM LARSOFT ***

In [2]:
data1,key_list = load_larsoft_root("RAW","data/opdetraw_hist.root",     "opdigiana",np.loadtxt("template/fbk_deco.txt"),label="RAW")
data2,key_list = load_larsoft_root("DEC","data/deconv_gauss_hist.root", "opdecoana",np.loadtxt("template/fbk_deco.txt"),label="GAUSS")
data3,key_list = load_larsoft_root("DEC","data/deconv_wiener_hist.root","opdecoana",np.loadtxt("template/fbk_deco.txt"),label="WIENER")

color_map={data1["LABEL"]: "#3366CC", data2["LABEL"]: "#66AA00", data3["LABEL"]:"#FF9900"}


In [3]:
SER = []
templates = os.listdir("template")

for path in templates:
    SER.append(np.loadtxt("template/"+path))

fig = go.Figure()
for i in range(len(SER)):
    fig.add_trace(go.Scatter(name=templates[i],x=np.arange(len(SER[i])),y=SER[i]))

fig.update_layout(title_text='Template comparison',xaxis_title  = "Time in [ticks]",yaxis_title  = "Amp. in [ADC]",)
fig.show()

*** VISUALIZE DATA ***

In [35]:
num = np.random.randint(len(data1["RECO"]["WVF"]))
# num = np.where(np.asarray(data2["RECO"]["CH"]) == 232)[0][0]
print(num)

fig = make_subplots(rows=1, cols=2,subplot_titles=('', ''))

fig.add_trace(go.Scatter(line=dict(color=color_map[data1["LABEL"]]),name="%s #PE: %.2f"%(data1["LABEL"],data1["RECO"]["PE"][num]),x=data1["RECO"]["WVF_X"][num],y=data1["RECO"]["WVF"][num]-data1["PEDESTAL"]),col=1,row=1)
fig.add_trace(go.Scatter(line=dict(color=color_map[data2["LABEL"]]),name="%s #PE: %.2f"%(data2["LABEL"],data2["RECO"]["PE"][num]),x=data2["RECO"]["WVF_X"][num],y=data2["RECO"]["WVF"][num]),col=2,row=1)
fig.add_trace(go.Scatter(line=dict(color=color_map[data3["LABEL"]]),name="%s #PE: %.2f"%(data3["LABEL"],data3["RECO"]["PE"][num]),x=data3["RECO"]["WVF_X"][num],y=data3["RECO"]["WVF"][num]),col=2,row=1)

fig.add_vline(x=data1["RECO"]["T0"][num], line_width=2, line_dash="dash", line_color="gray",col=1)
fig.add_vline(x=data2["RECO"]["T0"][num], line_width=2, line_dash="dash", line_color="gray",col=2)
fig.add_vline(x=data3["RECO"]["T0"][num], line_width=2, line_dash="dash", line_color="gray",col=2)

try:
    if len(data1["TRUE"]["PETIMES"][num]) > 100:
        fig.add_trace(go.Histogram(name="TRUE #PE: %.2f"%(len(data1["TRUE"]["PETIMES"][num])),x=np.asarray(data1["TRUE"]["PETIMES"][num])+data1["PRETRIGGER"]*data1["SAMPLING"]*1e6),col=1,row=1)
    else:
        fig.add_trace(go.Scatter(mode="markers",line=dict(color="#DC3912"),name="TRUE #PE: %.2f"%(len(data1["TRUE"]["PETIMES"][num])),x=np.asarray(data1["TRUE"]["PETIMES"][num])+data1["PRETRIGGER"]*data1["SAMPLING"]*1e6,y=np.zeros(len(data1["TRUE"]["PETIMES"][num]))),col=1,row=1)
        fig.add_trace(go.Scatter(mode="markers",line=dict(color="#DC3912"),name="TRUE #PE: %.2f"%(len(data1["TRUE"]["PETIMES"][num])),x=np.asarray(data1["TRUE"]["PETIMES"][num])+data1["PRETRIGGER"]*data1["SAMPLING"]*1e6,y=np.zeros(len(data1["TRUE"]["PETIMES"][num]))),col=2,row=1)
except:
    print("WVF WITHOUT TRUE PHOTONS")
    
fig.update_layout(title = "Deco. wvf comparison",xaxis_title  = "Time in [&mu;s]",yaxis_title  = "Amp. in [ADC]",xaxis2_title = "Time in [&mu;s]",yaxis2_title = "Amp. in [a.u.]",)
fig.update_layout(autosize=True,font=dict(size=16))

fig.show()

348


*** CONFIGURE FILTER FOR ANALYSIS ***

In [38]:
pe_error_1 = (np.asarray(data1["RECO"]["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
pe_error_2 = (np.asarray(data2["RECO"]["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])
pe_error_3 = (np.asarray(data3["RECO"]["PE"])-np.asarray(data1["TRUE"]["PE"]))/np.asarray(data1["TRUE"]["PE"])

t0_reco_1 = np.asarray(data1["RECO"]["T0"])-data1["PRETRIGGER"]*data1["SAMPLING"]*1e6
t0_reco_2 = np.asarray(data2["RECO"]["T0"])-data1["PRETRIGGER"]*data1["SAMPLING"]*1e6
t0_reco_3 = np.asarray(data3["RECO"]["T0"])-data1["PRETRIGGER"]*data1["SAMPLING"]*1e6

amp          = np.concatenate([data1["RECO"]["AMP"],data2["RECO"]["AMP"],data3["RECO"]["AMP"]])
pe_reco      = np.concatenate([data1["RECO"]["PE"],data2["RECO"]["PE"],data3["RECO"]["PE"]])
pe_true      = np.concatenate([data1["TRUE"]["PE"],data1["TRUE"]["PE"],data1["TRUE"]["PE"]])
pe_error     = 100*np.concatenate([pe_error_1,pe_error_2,pe_error_3])
t0_reco      = np.concatenate([t0_reco_1,t0_reco_2,t0_reco_3])
filter_label = np.concatenate([[data1["LABEL"]]*len(pe_error_1),[data2["LABEL"]]*len(pe_error_2),[data3["LABEL"]]*len(pe_error_3)])

df = pd.DataFrame({"FILTER":filter_label, "'%'ERROR PE":pe_error, "TRUE PE":pe_true, "RECO PE":pe_reco, "AMP [a.u.]":amp, "RECO T0":t0_reco})

In [39]:
fig = px.scatter(data_frame=df[(df['TRUE PE'] > 10)*(df['TRUE PE'] < 1000)], y='TRUE PE', x="RECO PE",
color='FILTER',color_discrete_map=color_map, marginal_x='histogram',trendline="ols")
fig.update_layout(autosize=False,width=1500,height=800,font=dict(size=16))
fig.show()

results = px.get_trendline_results(fig)
# print(results)
results.query("FILTER == 'GAUSS'").px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.557e+06
Date:                Sat, 11 Mar 2023   Prob (F-statistic):               0.00
Time:                        21:33:04   Log-Likelihood:                -1353.9
No. Observations:                 460   AIC:                             2712.
Df Residuals:                     458   BIC:                             2720.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3718      0.287      1.294      0.196      -0.193       0.936
x1             1.0019      0.001   1247.885      0.000       1.000       1.003
==============================================================================
Omnibus:                        2.734   Durbin-Watson:                   2.078
Prob(Omnibus):                  0.255   Jarque-Bera (JB):                2.888
Skew:                          -0.048   Prob(JB):                        0.236
Kurtosis:                       3.376   Cond. No.                         479.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [40]:
fig = px.scatter(data_frame=df[(df['TRUE PE'] > 10)*(df['TRUE PE'] < 2000)], y="TRUE PE", x="'%'ERROR PE",
color='FILTER',color_discrete_map=color_map,marginal_x='box',template="plotly")
# fig.update_layout(title_text='T0 recovery',xaxis_title  = "Time in [&mu;s]",yaxis_title  = "Counts")
fig.update_layout(autosize=False,width=1500,height=800,font=dict(size=16))
fig.update_traces(opacity=0.75)
fig.show()

In [41]:
fig = px.histogram(data_frame=df[(df['TRUE PE'] > 0)*(df['TRUE PE'] < 1000)], x="RECO T0",
color='FILTER',color_discrete_map=color_map,barmode="group",marginal='box',template="plotly",nbins=3000,range_x=[-0.16,0.32])
# fig.update_layout(title_text='T0 recovery',xaxis_title  = "Time in [&mu;s]",yaxis_title  = "Counts")
fig.update_layout(autosize=False,width=1500,height=800,font=dict(size=16))

fig.show()